<a href="https://colab.research.google.com/github/dmeoli/NeuroSAT/blob/master/GATQSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [ ]:
import os

if not os.path.isdir('neuroSAT'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuroSAT
  %cd neuroSAT
else:
  %cd neuroSAT
  !git pull

/content/gdrive/My Drive/neuroSAT
Already up to date.


In [ ]:
!pip install -r requirements.txt

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html, https://data.pyg.org/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 1.6 MB 7.7 MB/s 
     |████████████████████████████████| 124 kB 91.2 MB/s 
     |██████████████▋                 | 834.1 MB 1.3 MB/s eta 0:12:50tcmalloc: large alloc 1147494400 bytes == 0x5611c38c8000 @  0x7f55de640615 0x5611c00884cc 0x5611c016847a 0x5611c008b2ed 0x5611c017ce1d 0x5611c00fee99 0x5611c00f99ee 0x5611c008cbda 0x5611c00fed00 0x5611c00f99ee 0x5611c008cbda 0x5611c00fb737 0x5611c017dc66 0x5611c00fadaf 0x5611c017dc66 0x5611c00fadaf 0x5611c017dc66 0x5611c00fadaf 0x5611c008d039 0x5611c00d0409 0x5611c008bc52 0x5611c00fec25 0x5611c00f99ee 0x5611c008cbda 0x5611c00fb737 0x5611c00f99ee 0x5611c008cbda 0x5611c00fa915 0x5611c008cafa 0x5611c00fac0d 0x5611c00f99ee
     |██████████████████▌             | 1055.7 MB 1.2 MB/s eta 0:10:34tcmalloc: large alloc 1434370048 bytes == 0x561207f1e000 @  0x7f55de640615 0x5611c00884cc 0x5611

In [ ]:
datasets = {'uniform-random-3-sat': {'train': ['uf50-218', 'uuf50-218',
                                               'uf100-430', 'uuf100-430'],
                                     'val': ['uf50-218', 'uuf50-218',
                                             'uf100-430', 'uuf100-430'],
                                     'inner_test': ['uf50-218', 'uuf50-218',
                                                    'uf100-430', 'uuf100-430'],
                                     'test': ['uf250-1065', 'uuf250-1065']},
            'graph-coloring': {'train': ['flat50-115'],
                               'val': ['flat50-115'],
                               'inner_test': ['flat50-115'],
                               'test': ['flat30-60',
                                        'flat75-180',
                                        'flat100-239',
                                        'flat125-301',
                                        'flat150-360',
                                        'flat175-417',
                                        'flat200-479']}}

In [ ]:
%cd GQSAT

/content/gdrive/My Drive/neuroSAT/GQSAT


## Build C++

In [ ]:
%cd minisat
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuroSAT/GQSAT/minisat
rm -f   build/release/minisat/core/Solver.o  build/release/minisat/core/Main.o  build/release/minisat/simp/SimpSolver.o  build/release/minisat/simp/Main.o  build/release/minisat/utils/System.o  build/release/minisat/utils/Options.o  build/release/minisat/gym/GymSolver.o   build/debug/minisat/core/Solver.o  build/debug/minisat/core/Main.o  build/debug/minisat/simp/SimpSolver.o  build/debug/minisat/simp/Main.o  build/debug/minisat/utils/System.o  build/debug/minisat/utils/Options.o  build/debug/minisat/gym/GymSolver.o   build/profile/minisat/core/Solver.o  build/profile/minisat/core/Main.o  build/profile/minisat/simp/SimpSolver.o  build/profile/minisat/simp/Main.o  build/profile/minisat/utils/System.o  build/profile/minisat/utils/Options.o  build/profile/minisat/gym/GymSolver.o   build/dynamic/minisat/core/Solver.o  build/dynamic/minisat/core/Main.o  build/dynamic/minisat/simp/SimpSolver.o  build/dynamic/minisat/simp/Main.o  build/dynamic/m

## Uniform Random 3-SAT

We split *(u)uf50-218* and *(u)uf100-430* into three subsets: 800 training problems, 100 validation, and 100 test problems.

For generalization experiments, we use 100 problems from all the other benchmarks.

To evaluate the knowledge transfer properties of the trained models across different task families, we use 100 problems from all the *graph coloring* benchmarks.

In [ ]:
PROBLEM_TYPE='uniform-random-3-sat'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

### Add metadata for evaluation (train and validation set)

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME"
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME"

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0 \
    --use_attention \
    --heads 3

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME"

for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test'] + datasets[PROBLEM_TYPE]['kt_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

### Evaluate

In [ ]:
models = {'uf50-218': ('Nov12_14-06-54_c42e8ad320d8', 
                       'model_50003'),
          'uuf50-218': ('Nov12_20-35-32_c42e8ad320d8', 
                        'model_50006'),
          'uf100-430': ('Nov13_03-55-51_c42e8ad320d8', 
                        'model_50085'),
          'uuf100-430': ('Nov14_03-26-36_54337a27a809', 
                         'model_50003')}

We test these trained models on the inner test sets.

In [ ]:
for SAT_MODEL in models.keys():
  for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
    # do not use models trained on unSAT problems to solve SAT ones
    if not (SAT_MODEL.startswith('uuf') and PROBLEM_NAME.startswith('uf')):
      for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
        MODEL_DIR = models[SAT_MODEL][0]
        CHECKPOINT = models[SAT_MODEL][1]
        !python evaluate.py \
          --logdir log \
          --env-name sat-v0 \
          --core-steps -1 \
          --eps-final 0.0 \
          --eval-time-limit 100000000000000 \
          --no_restarts \
          --test_time_max_decisions_allowed "$MODEL_DECISION" \
          --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME" \
          --model-dir runs/"$MODEL_DIR" \
          --model-checkpoint "$CHECKPOINT".chkp \
          >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

We test the trained models on the outer test sets.

In [ ]:
for SAT_MODEL in models.keys():
  for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
    # do not use models trained on unSAT problems to solve SAT ones
    if not (SAT_MODEL.startswith('uuf') and PROBLEM_NAME.startswith('uf')):
      for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
        MODEL_DIR = models[SAT_MODEL][0]
        CHECKPOINT = models[SAT_MODEL][1]
        !python evaluate.py \
          --logdir log \
          --env-name sat-v0 \
          --core-steps -1 \
          --eps-final 0.0 \
          --eval-time-limit 100000000000000 \
          --no_restarts \
          --test_time_max_decisions_allowed "$MODEL_DECISION" \
          --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
          --model-dir runs/"$MODEL_DIR" \
          --model-checkpoint "$CHECKPOINT".chkp \
          >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

We test the trained models on the *graph coloring* test sets, both inners and outers.

In [ ]:
for SAT_MODEL in models.keys():
  # do not use models trained on unSAT problems to solve SAT ones
  if not SAT_MODEL.startswith('uuf'):
    for PROBLEM_NAME in datasets['graph-coloring']['inner_test']:
      for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
        MODEL_DIR = models[SAT_MODEL][0]
        CHECKPOINT = models[SAT_MODEL][1]
        !python evaluate.py \
          --logdir log \
          --env-name sat-v0 \
          --core-steps -1 \
          --eps-final 0.0 \
          --eval-time-limit 100000000000000 \
          --no_restarts \
          --test_time_max_decisions_allowed "$MODEL_DECISION" \
          --eval-problems-paths ../data/graph-coloring/test/"$PROBLEM_NAME" \
          --model-dir runs/"$MODEL_DIR" \
          --model-checkpoint "$CHECKPOINT".chkp \
          >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

In [ ]:
for SAT_MODEL in models.keys():
  # do not use models trained on unSAT problems to solve SAT ones
  if not SAT_MODEL.startswith('uuf'):
    for PROBLEM_NAME in datasets['graph-coloring']['test']:
      for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
        MODEL_DIR = models[SAT_MODEL][0]
        CHECKPOINT = models[SAT_MODEL][1]
        !python evaluate.py \
          --logdir log \
          --env-name sat-v0 \
          --core-steps -1 \
          --eps-final 0.0 \
          --eval-time-limit 100000000000000 \
          --no_restarts \
          --test_time_max_decisions_allowed "$MODEL_DECISION" \
          --eval-problems-paths ../data/graph-coloring/"$PROBLEM_NAME" \
          --model-dir runs/"$MODEL_DIR" \
          --model-checkpoint "$CHECKPOINT".chkp \
          >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

## Graph Coloring

Graph coloring benchmarks have only 100 problems each, except for *flat50-115* which contains 1000, so we split it into three subsets: 800 training problems, 100 validation, and 100 test problems.

For generalization experiments, we use 100 problems from all the other benchmarks.

In [ ]:
PROBLEM_TYPE='graph-coloring'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

### Add metadata for evaluation (train and validation set)

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME"
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME"

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 0.1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0 \
    --use_attention \
    --heads 3

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME"

for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

### Evaluate

In [ ]:
MODEL_DIR='Dec09_12-16-16_d4e65e7af705'
CHECKPOINT='model_50001'

We test this trained model on the inner test set.

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

We test the trained model on the outer test sets.

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv